In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [27]:
dftweets = pd.read_csv('../../data/SPX.csv', parse_dates = True)
dftweets = dftweets[['date' , 'content', 'likeCount']]
#dftweets = dftweets[dftweets['likeCount'] >2]

In [28]:
dftweets.shape

(238768, 3)

In [29]:
dftweets

,date,content,likeCount
0,2019-05-31,Major world indices:\n$SPX: 2751.51\n$FTSE: 71...,1
1,2019-05-31,Yesterday - Industry: #SOFTWARE &amp; SERVICES...,1
2,2019-05-31,$SPX Monthly. Big red outside bar closing on t...,1
3,2019-05-31,@fintwitchair @OptionsAction $SPX is my prefer...,1
4,2019-05-31,$SPX weekly https://t.co/mnTD7440QV,4
...,...,...,...
238763,2018-06-01,"Tuesday night ➡️ mkt was ⬇️significantly, I wa...",3
238764,2018-06-01,there are no such monthly signs in the states ...,2
238765,2018-06-01,The Trump tax cuts aren’t trickling down yet h...,1
238766,2018-06-01,"In the first quarter of the year, S&amp;P 500 ...",19


In [30]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
import string 

def processTweet(tweet):
    tweet = re.sub(r'\&\w*;', '', tweet)
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    tweet = re.sub(r'#\w*', '', tweet)
    tweet = re.sub(r'[' + string.punctuation.replace('@', '') + ']+', ' ', tweet)
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet = tweet.lstrip(' ') 
    tweet = re.sub('https', ' ', tweet)
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

x = list(dftweets['content'])
for i in range(0,len(x)):
  x[i] = processTweet(x[i])

dftweets['content'] = x

tokenizer = RegexpTokenizer(r'\w+')
regexptokenized = []
for i in list(dftweets['content']):
  regexptokenized.append(tokenizer.tokenize(i))
#tokenizedcontent = tokenizer.tokenize(list(dftweets['content']))
print(regexptokenized[0:5])
tokenized = []


filtered_words = [word for word in regexptokenized if word not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emu4y\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emu4y\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[['major', 'world', 'indices', '2751', '7152', '11706', '20424', 'more'], ['yesterday', 'industry', 'services', 'was', 'positive', 'with', 'applic', 'positive', 'stocks', 'was', 'positive', 'with', 'positive'], ['monthly', 'big', 'red', 'outside', 'bar', 'closing', 'the', 'lows', 'haven', 'seen', 'those', 'while', '20m', 'sma', 'still', 'sloping', 'that', 'about', 'the', 'only', 'positive', '2600', 'sticks', 'out', 'depending', 'timing', 'that', 'would', 'correspond', 'lower', 'monthly', 'similar', 'dec', 'low'], ['preference', 'you', 'are', 'more', 'newbie', 'with', 'until', 'you', 'mastered', 'the', 'signals'], ['weekly']]


In [31]:
print(tokenized[0:5])
print(filtered_words[0:5])

[]
[['major', 'world', 'indices', '2751', '7152', '11706', '20424', 'more'], ['yesterday', 'industry', 'services', 'was', 'positive', 'with', 'applic', 'positive', 'stocks', 'was', 'positive', 'with', 'positive'], ['monthly', 'big', 'red', 'outside', 'bar', 'closing', 'the', 'lows', 'haven', 'seen', 'those', 'while', '20m', 'sma', 'still', 'sloping', 'that', 'about', 'the', 'only', 'positive', '2600', 'sticks', 'out', 'depending', 'timing', 'that', 'would', 'correspond', 'lower', 'monthly', 'similar', 'dec', 'low'], ['preference', 'you', 'are', 'more', 'newbie', 'with', 'until', 'you', 'mastered', 'the', 'signals'], ['weekly']]


In [32]:
stringcontent = ""
newcontent = []
for i in filtered_words:
  stringcontent = ""
  for j in i:
    stringcontent += j + " " 
  newcontent.append(stringcontent)
dftweets['content'] = newcontent

In [33]:
dftweets.head()

,date,content,likeCount
0,2019-05-31,major world indices 2751 7152 11706 20424 more,1
1,2019-05-31,yesterday industry services was positive with ...,1
2,2019-05-31,monthly big red outside bar closing the lows h...,1
3,2019-05-31,preference you are more newbie with until you ...,1
4,2019-05-31,weekly,4


In [34]:
#Sentiment Analysis
!pip install nltk
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


positive = []
negative = []
neutral = []
mood = []

sid = SentimentIntensityAnalyzer()
sentiment = []
for i in list(dftweets['content']):
  sentiment.append(sid.polarity_scores(i))
  if sid.polarity_scores(i)['compound'] >= 0.1:
    positive.append(i)
    mood.append('positive')
  elif sid.polarity_scores(i)['compound'] <= -0.1:
    negative.append(i)
    mood.append('negative')
  else:
    neutral.append(i)
    mood.append('neutral')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\emu4y\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [35]:
neg = []
pos = []
neu = []
compound = []
for i in sentiment:
  neg.append(i['neg'])
  pos.append(i['pos'])
  neu.append(i['neu'])
  compound.append(i['compound'])
dftweets['negative'] = neg
dftweets['pos'] = pos
dftweets['neutral'] = neu
dftweets['compound'] = compound
dftweets['mood'] = mood

In [36]:
dftweets.head()

,date,content,likeCount,negative,pos,neutral,compound,mood
0,2019-05-31,major world indices 2751 7152 11706 20424 more,1,0.000,0.000,1.000,0.0000,neutral
1,2019-05-31,yesterday industry services was positive with ...,1,0.000,0.615,0.385,0.9371,positive
2,2019-05-31,monthly big red outside bar closing the lows h...,1,0.154,0.091,0.756,-0.1280,negative
3,2019-05-31,preference you are more newbie with until you ...,1,0.000,0.000,1.000,0.0000,neutral
4,2019-05-31,weekly,4,0.000,0.000,1.000,0.0000,neutral


In [37]:
dftweets.to_csv('../../data/SPXprocessed.csv')

In [ ]:
# Check lag time between tweets and change in price + importance value
# Filter out people w more followers (more followers / gov realtion generally have higher importance)
# classify tweets that increase index and decrease index (training data)
# group tweets by day, then give a score (positive / negative) then take average and then compare to the ACTUAL s&p score (either by percent or say index moved up or down)